# CS 513 Winery Data Cleaning

## 1. Import and Read CSV

Prerequisite library: pandas

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
winery = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

In [3]:
winery.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## 2. Trim appropriate string data

In [5]:
winery['designation'] = winery['designation'].str.strip()
winery['taster_twitter_handle'] = winery['taster_twitter_handle'].str.strip()
winery['title'] = winery['title'].str.strip()
winery['variety'] = winery['variety'].str.strip()
winery['winery'] = winery['winery'].str.strip()

## 3. Transform numeric data

In [4]:
winery['points'] = winery['points'].astype(int)
winery['price'] = winery['price'].astype(float)

## 4. Clean numeric data

In [6]:
winery.price.describe()

count    120975.000000
mean         35.363389
std          41.022218
min           4.000000
25%          17.000000
50%          25.000000
75%          42.000000
max        3300.000000
Name: price, dtype: float64

### Imputation

Impute price data based on groupby of 3 other columns for more analysis.

In [7]:
winery['price_imputeCountry'] = winery['price'].fillna(
    winery.groupby('country')['price'].transform('median'))
winery['price_imputeProvince'] = winery['price'].fillna(
    winery.groupby('province')['price'].transform('median'))
winery['price_imputeVariety'] = winery['price'].fillna(
    winery.groupby('variety')['price'].transform('median'))

### Outlier Removal

In [8]:
cutoff = winery.price.quantile(.99)
print(cutoff)

155.0


In [9]:
winery['price'] = winery['price'].fillna(round(winery['price'].mean()))
winery = winery[winery['price']<=cutoff]

In [10]:
winery.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,price_imputeCountry,price_imputeProvince,price_imputeVariety
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,28.0,22.0,19.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,15.0,15.0,15.0
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,14.0,14.0,14.0
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,13.0,13.0,13.0
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,65.0,65.0,65.0


In [11]:
winery.price.describe()

count    128788.000000
mean         32.962924
std          22.130144
min           4.000000
25%          18.000000
50%          28.000000
75%          40.000000
max         155.000000
Name: price, dtype: float64

## 5. Check for ICV

### Remove Duplicates

In [12]:
sum(winery.duplicated(keep=False))

19892

In [13]:
winery = winery.drop_duplicates(keep='first')
winery.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,price_imputeCountry,price_imputeProvince,price_imputeVariety
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,28.0,22.0,19.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,15.0,15.0,15.0
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,14.0,14.0,14.0
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,13.0,13.0,13.0
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,65.0,65.0,65.0


### Remove NAs

NAs for designation, region_1, region_2, taster_twitter_handle are unavoidable in order to maintain size of dataset. This is fine as those labels are potentially useful but not necessary for analysis. However, more immediately relevant and smaller counts of NA can be dropped or imputed. Taster name can be filled with "Anonymous" as the reviews should still be valid. Country and province could be replaced manually with research, but for the purpose of efficiency wil be dropped. The generated price columns can be further imputed.

In [14]:
winery.isna().sum()

country                     59
description                  0
designation              34227
points                       0
price                        0
province                    59
region_1                 19439
region_2                 72330
taster_name              24726
taster_twitter_handle    29216
title                        0
variety                      1
winery                       0
price_imputeCountry          5
price_imputeProvince         7
price_imputeVariety         10
dtype: int64

In [15]:
winery = winery.dropna(subset=['country','province','variety'])
winery['taster_name'] = winery['taster_name'].fillna('Anonymous')

In [16]:
winery['price_imputeCountry'] = winery['price_imputeCountry'].fillna(round(winery['price_imputeCountry'].mean()))
winery['price_imputeProvince'] = winery['price_imputeProvince'].fillna(round(winery['price_imputeProvince'].mean()))
winery['price_imputeVariety'] = winery['price_imputeVariety'].fillna(round(winery['price_imputeVariety'].mean()))

In [17]:
winery.isna().sum()

country                      0
description                  0
designation              34216
points                       0
price                        0
province                     0
region_1                 19379
region_2                 72270
taster_name                  0
taster_twitter_handle    29215
title                        0
variety                      0
winery                       0
price_imputeCountry          0
price_imputeProvince         0
price_imputeVariety          0
dtype: int64

In [18]:
winery.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,price_imputeCountry,price_imputeProvince,price_imputeVariety
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,28.0,22.0,19.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,15.0,15.0,15.0
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,14.0,14.0,14.0
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,13.0,13.0,13.0
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,65.0,65.0,65.0
